In [24]:
import networkx as nx
import numpy as np
# Note: scipy.optimize.linprog and scipy.stats are no longer needed for baseline
# from scipy.optimize import linprog
# from scipy.stats import norm # If norm was used elsewhere, keep it. Poisson is in experiment file.
from copy import deepcopy
import os
import logging # Keep logging import

In [25]:
# --- Basic Logging Setup ---
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger()
# --- End Logging Setup ---

In [26]:
image_mode = globals().get('image_mode', False)
debug_mode = globals().get('debug_mode', False)

In [27]:
%run attack_graph_MARA.ipynb

In [28]:
# Assume logger is configured externally (in the experiment file)
logger = logging.getLogger()

# Global Default Weight Variable (Keep as it might be used in graph processing)
DEFAULT_WEIGHT_VALUE = 0

In [29]:
# --- Graph Pre-Processing Functions (Keep Unchanged) ---

def find_and_add_entry_node(graph):
    original_roots = [n for n, deg in graph.in_degree() if deg == 0]
    if len(original_roots) > 1:
        entry = 0
        graph.add_node(entry)
        for r in original_roots:
            graph.add_edge(entry, r, weight=DEFAULT_WEIGHT_VALUE)
        return entry, graph, original_roots
    else:
        entry = original_roots[0] if original_roots else None # Handle case with no roots
        if entry is None:
             raise ValueError("Graph has no root nodes.")
        return entry, graph, original_roots

In [30]:
def merge_targets_with_multi_edges(orig_graph):
    targets = [n for n, deg in orig_graph.out_degree() if deg == 0]
    if len(targets) <= 1:
        # If 0 or 1 target, find the single target node if it exists
        single_target = targets[0] if targets else None
        return orig_graph, single_target # Return graph and the single target

    merged_label = "c(" + ",".join(str(t) for t in targets) + ")"
    newG = nx.MultiDiGraph()
    non_targets = [node for node in orig_graph.nodes() if node not in targets]

    for node in non_targets:
        newG.add_node(node)
    newG.add_node(merged_label)

    pred_target_edges = {}
    for u, v, data in orig_graph.edges(data=True):
        if v in targets:
            if u not in pred_target_edges:
                pred_target_edges[u] = []
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)
            pred_target_edges[u].append((weight, v))

    for u, edges in pred_target_edges.items():
        weight_counts = {}
        for weight, _ in edges:
            weight_counts[weight] = weight_counts.get(weight, 0) + 1
        for weight, count in weight_counts.items():
            for _ in range(count):
                newG.add_edge(u, merged_label, weight=weight)

    for u, v, data in orig_graph.edges(data=True):
        if v not in targets and u not in targets:
            newG.add_edge(u, v, **data)

    return newG, merged_label # Return graph and the merged target label

# --- Elements preparation for the Game (Keep generate_game_elements) ---
# We still need it to find potential defense locations (as1) and the target node.
# We will ignore the 'as2' (all paths) it returns for the baseline.

In [31]:
def generate_game_elements(graph, entry_node, original_roots):
    """
    Sets up elements needed. For baseline, we mainly need as1 and target_list.
    as2 (all paths) and other elements related to path probabilities are ignored later.
    """
    target_list = [n for n,d in graph.out_degree() if d == 0]
    if len(target_list) != 1:
         # If merge_targets was called, there should be exactly one.
         # If not (e.g., graph initially had 1 target), this is okay.
         if not target_list:
              raise ValueError("Graph has no target nodes after processing.")
         # Use the first target if multiple somehow exist post-merge (shouldn't happen)
         logger.warning(f"Expected one target node after potential merge, found: {target_list}. Using {target_list[0]}")

    # Find paths just to identify nodes involved (V)
    # Note: Using the graph directly might be simpler if V is just needed for as1
    try:
        # Ensure entry and target are in the graph before finding paths
        if not graph.has_node(entry_node):
             raise nx.NodeNotFound(f"Entry node {entry_node} not in graph.")
        if not graph.has_node(target_list[0]):
             raise nx.NodeNotFound(f"Target node {target_list[0]} not in graph.")

        # Check connectivity
        if not nx.has_path(graph, entry_node, target_list[0]):
            logger.warning(f"No path exists between entry {entry_node} and target {target_list[0]}.")
            # Handle this case: maybe return empty as1/as2 or raise error?
            # For baseline, if no path, attacker success is 0.
            V = set([entry_node, target_list[0]]) # Minimal V
            as1 = []
            as2 = [] # No paths
            node_order = list(V)
            adv_list = []
            theta = {}
            m = 0

        else:
            # Proceed with finding paths if connected
            raw_routes = list(nx.all_simple_paths(graph, entry_node, target_list[0]))

            consolidated_routes = []
            seen_paths = set()
            for path in raw_routes:
                path_key = tuple(path)
                if path_key not in seen_paths:
                    seen_paths.add(path_key)
                    consolidated_routes.append(list(path))
            routes = consolidated_routes
            as2 = routes # Keep for compatibility, but baseline attacker ignores this list

            V_set = set(node for path in routes for node in path)
            V = sorted(list(V_set), key=str) # Keep V for defining as1 scope

            # Get nodes in topological order if needed elsewhere, otherwise optional for baseline
            try:
                topo_all = list(nx.topological_sort(graph))
                node_order = [n for n in topo_all if n in V_set]
            except nx.NetworkXUnfeasible: # Handle cycles if graph isn't a DAG
                logger.warning("Graph contains cycles, topological sort not possible. Node order may be arbitrary.")
                node_order = V # Fallback to sorted list

            # Calculate as1: Potential defense locations (excluding entry, target, original roots)
            excluded = {entry_node} | set(target_list) | set(original_roots)
            as1 = [n for n in V if n not in excluded]

            # Calculate adv_list (intermediate nodes) - Not strictly needed for baseline outcome calc
            excluded_nodes = {entry_node} | set(target_list)
            adv_list = [n for n in V if n not in excluded_nodes]
            theta = {loc: 1/len(adv_list) for loc in adv_list} if adv_list else {} # Not used in baseline calc
            m = len(routes) # Not used in baseline calc

    except nx.NodeNotFound as e:
         logger.error(f"Error finding game elements: {e}")
         # Decide how to handle - maybe return defaults indicating failure?
         V, as1, as2, target_list, node_order, adv_list, theta, m = [], [], [], [], [], [], {}, 0
         raise # Re-raise the exception or handle gracefully

    # Return all elements for potential compatibility, though only as1/target needed for baseline core logic
    return routes, V, as1, as2, target_list, node_order, adv_list, theta, m


# --- Calculate Payoffs based on the game elements (REMOVED/REPLACED for Baseline) ---
# The functions lossDistribution and calculate_payoff_distribution are no longer needed.

# --- Find the Equilibrium (REMOVED/REPLACED for Baseline) ---
# The function solve_game (using linprog) is no longer needed.

# --- Debug Code / Helpers (Can Keep If Useful) ---

In [32]:
def print_debug_info(graph, stage=""):
    # (Keep implementation as provided in the original code)
    print(f"\n{stage}:")
    print("Nodes:", list(graph.nodes()))
    print("Total list of Edges with their weights:")
    if isinstance(graph, nx.MultiDiGraph):
        for u, v, key, data in graph.edges(data=True, keys=True):
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)
            print(f"{u} -> {v} (key={key}) : {weight}")
    else:
        for u, v, data in graph.edges(data=True):
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)
            print(f"{u} -> {v} : {weight}")



In [33]:
# (Keep reorder_strategies and debug_paths if they are helpful for debugging setup,
# but they are not core to the baseline calculation itself)
def reorder_strategies(as1, as2):
     # (Keep implementation as provided)
     pass
def debug_paths(as1, as2):
     # (Keep implementation as provided)
     pass

In [34]:
def calculate_single_payoff_distribution(graph, check_node, path, V, adv_list, theta,
                                         random_steps_fn, attack_rate, defense_rate, node_order):
    """
    Calculates the final probability distribution U for a SINGLE pair of:
    - check_node: The node the defender checks (can be None if no check active)
    - path: The specific attack path the attacker uses.

    Uses the original avatar logic (theta) and probabilistic movement (random_steps_fn).
    Returns the final normalized probability vector U ordered by node_order.
    Returns None if calculation fails.
    """
    try:
        U = np.zeros(len(V)) # Initialize probability vector for all nodes in V

        # Map node names to indices in V for quick lookup
        V_indices = {node: i for i, node in enumerate(V)}

        # --- Loop through each possible attacker starting position (avatar) ---
        for avatar in adv_list:
            L = np.zeros(len(V)) # Temp vector for this avatar's outcome

            avatar_start_index_in_V = V_indices.get(avatar)
            if avatar_start_index_in_V is None:
                logger.warning(f"Avatar {avatar} not in V {V}. Skipping.")
                continue # Should not happen if adv_list is subset of V

            # --- Case 1: Avatar starts ON the given attack path ---
            if avatar in path:
                start_idx_in_path = path.index(avatar)
                route_from_avatar = path[start_idx_in_path:]

                if not route_from_avatar: # Path starting at avatar is empty?
                    L[avatar_start_index_in_V] = 1.0 # Stays at start
                else:
                    # Get probabilistic movement distribution along this route_from_avatar
                    pdf_d = random_steps_fn(route_from_avatar, attack_rate, defense_rate, graph)

                    # Determine the cut point based on the defender's check_node
                    cutPoint = len(route_from_avatar) # Default: no cut
                    if check_node is not None and check_node in route_from_avatar:
                        check_idx_in_route = route_from_avatar.index(check_node)
                        cutPoint = check_idx_in_route + 1 # Include check node, stop after

                    # Take probabilities up to the cut point
                    pdf_subset = pdf_d[:cutPoint]
                    route_subset = route_from_avatar[:cutPoint]

                    # Renormalize the subset probabilities
                    payoffDistr = np.zeros_like(pdf_subset)
                    subset_sum = np.sum(pdf_subset)
                    if subset_sum < 1e-15:
                        # If sum is zero, means attacker definitely stops at the last node before cut
                        if len(payoffDistr) > 0:
                            payoffDistr[-1] = 1.0
                    else:
                        payoffDistr = pdf_subset / subset_sum

                    # Assign renormalized probabilities to the corresponding nodes in L
                    for idx_node, node in enumerate(route_subset):
                         node_index_in_V = V_indices.get(node)
                         if node_index_in_V is not None:
                              L[node_index_in_V] = payoffDistr[idx_node]
                         else:
                              logger.warning(f"Node {node} from path not in V. Skipping assignment.")


            # --- Case 2: Avatar starts OFF the given attack path ---
            else:
                 # Attacker stays at the starting location 'avatar' with probability 1
                 L[avatar_start_index_in_V] = 1.0

            # --- Add weighted contribution to the final U vector ---
            if avatar in theta:
                U += theta[avatar] * L
            else:
                logger.warning(f"Avatar {avatar} not found in theta distribution {theta}. Skipping contribution.")


        # --- Final Normalization and Ordering ---
        U_sum = np.sum(U)
        if U_sum < 1e-15:
            # Avoid division by zero; assign uniform small probability? Or zero?
            # Assigning zero might be more appropriate if no avatar contributes.
            # Or maybe return None to indicate failure/no probability mass?
            # Let's return None for now.
             logger.warning(f"Sum of U is near zero for check={check_node}, path={path}. Calculation failed?")
             return None
            # Alternative: U = np.full_like(U, 1e-9) # Tiny uniform probability
            # U = U / np.sum(U)
        else:
            U = U / U_sum # Normalize
            # Ensure no zeros, replace with small epsilon (as in original code)
            U = np.where(U < 1e-7, 1e-7, U)
            # Renormalize again after adding epsilon
            U = U / np.sum(U)

        # Reorder U according to node_order
        # Create mapping from node_order to current U indices
        current_V_ordering = V # The order U is currently in
        map_to_current_indices = {node: idx for idx, node in enumerate(current_V_ordering)}

        U_reordered = np.zeros_like(U)
        for i, node in enumerate(node_order):
            if node in map_to_current_indices:
                 original_index = map_to_current_indices[node]
                 U_reordered[i] = U[original_index]
            else:
                 # Node in node_order not found in V used for calculation? Should not happen.
                 logger.warning(f"Node {node} from node_order not in V during reordering.")
                 # Handle error? Assign 0? For now, assign 0.
                 U_reordered[i] = 0.0

        # Final check? Renormalize U_reordered? Should still sum to 1 if mapping was complete.
        # final_sum = np.sum(U_reordered)
        # if not np.isclose(final_sum, 1.0):
        #      logger.warning(f"Reordered U sum is {final_sum}, renormalizing.")
        #      U_reordered = U_reordered / final_sum

        return U_reordered

    except Exception as e:
        logger.error(f"Error in calculate_single_payoff_distribution for check={check_node}, path={path}: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None # Indicate failure

In [35]:
from scipy.stats import poisson # Make sure scipy is installed
import numpy as np

# Define the random_steps function (e.g., using Poisson as in original experiment)
def random_steps(route, attack_rate=2, defense_rate=None, graph=None):
     """ Calculates step probability based on Poisson distribution. """
     length = len(route)
     if length == 0: return np.array([]) # Handle empty route
     # Ensure attack_rate is non-negative
     effective_attack_rate = max(0, attack_rate)
     pmf = poisson.pmf(np.arange(length), effective_attack_rate)
     pmf_sum = pmf.sum()
     if pmf_sum < 1e-15:
         # If sum is zero, probability is concentrated at step 0 (staying put)
         pmf_normalized = np.zeros(length)
         if length > 0:
             pmf_normalized[0] = 1.0
         return pmf_normalized
     else:
         pmf_normalized = pmf / pmf_sum
         # Optional: Handle potential NaN/Inf if pmf_sum was somehow zero despite check
         pmf_normalized = np.nan_to_num(pmf_normalized)
         return pmf_normalized

# Make sure run_game uses this by passing it in the final call,
# or rely on the internal default within the modified run_game.
# Best practice: pass it explicitly in the __main__ block.

In [36]:
def run_game(attacker_graph, defender_graph=None,
             attack_rate_list=None, # Used by random_steps_fn
             dropped=None,
             defense_rate_list=None,# Used by random_steps_fn
             random_steps_fn=None): # Crucial for probabilistic movement
    """
    MODIFIED FOR REVISED BASELINE CALCULATION.

    Calculates the attacker's expected success probability assuming:
    1. Attacker *always* uses the shortest path (in terms of number of edges).
    2. Defender applies a uniform random defense strategy over possible locations (as1).
    3. Considers avatar starting positions (theta) and probabilistic movement (random_steps_fn).

    Returns a dictionary mimicking the equilibrium output structure.
    """
    if defender_graph is None:
        defender_graph = attacker_graph
    if random_steps_fn is None:
        # Provide a default if none is passed (e.g., from standalone execution)
        # This default should match the experiment context, e.g., Poisson(lambda=2)
        from scipy.stats import poisson
        default_lambda = 2 # Example default, adjust as needed
        def default_random_steps(route, attack_rate=default_lambda, defense_rate=None, graph=None):
             length = len(route)
             if length == 0: return np.array([])
             pmf = poisson.pmf(np.arange(length), attack_rate)
             # Handle sum zero case (e.g., length 1, attack_rate very low)
             pmf_sum = pmf.sum()
             if pmf_sum < 1e-15:
                  # Assign full probability to the starting node
                  # This might need refinement based on desired behavior for zero-length paths
                  pmf_normalized = np.zeros(length)
                  if length > 0:
                       pmf_normalized[0] = 1.0
                  return pmf_normalized
             else:
                  pmf_normalized = pmf / pmf_sum
                  return pmf_normalized
        random_steps_fn = default_random_steps
        logger.warning(f"random_steps_fn not provided, using default Poisson(lambda={default_lambda})")

    # Use the first rate from the lists if provided, otherwise default (e.g., 0 or specific value)
    # These rates are passed to random_steps_fn
    attackRate = attack_rate_list[0] if attack_rate_list else 2 # Default attack rate if list empty/None
    defenseRate = defense_rate_list[0] if defense_rate_list else 0 # Default defense rate

    # --- Graph Pre-processing (Attacker Graph for pathfinding) ---
    graph_for_pathfinding = attacker_graph.copy()
    try:
        atk_virtual_entry_node, graph_for_pathfinding, atk_original_roots = find_and_add_entry_node(graph_for_pathfinding)
        graph_for_pathfinding, atk_merged_target = merge_targets_with_multi_edges(graph_for_pathfinding)
        if atk_merged_target is None:
             potential_targets = [n for n, deg in graph_for_pathfinding.out_degree() if deg == 0]
             if not potential_targets: raise ValueError("Attacker graph has no target node.")
             atk_target_node = potential_targets[0]
        else:
             atk_target_node = atk_merged_target
    except Exception as e:
        logger.error(f"Error during attacker graph pre-processing: {e}")
        return None

    # --- Graph Pre-processing (Defender Graph for defense locations 'as1') ---
    # Use the original attacker graph if defender_graph is None
    graph_for_defense_locs = defender_graph.copy()
    try:
        def_virtual_entry_node, graph_for_defense_locs, def_original_roots = find_and_add_entry_node(graph_for_defense_locs)
        graph_for_defense_locs, _ = merge_targets_with_multi_edges(graph_for_defense_locs)
    except Exception as e:
        logger.error(f"Error during defender graph pre-processing: {e}")
        return None

    # --- Generate Game Elements ---
    try:
        # Get all elements, we need as1, V, adv_list, theta, node_order
        # Pass attacker graph for elements related to paths/nodes (V, adv_list, theta, node_order)
        # Pass defender graph info for as1 calculation
        _, V, as1, all_paths, _, node_order, adv_list, theta, _ = generate_game_elements(
            graph_for_pathfinding, atk_virtual_entry_node, atk_original_roots
        )
        # We need to recalculate as1 based on the defender graph if it differs
        # (Generate elements call above used attacker graph for V, this affects as1)
        # Re-run just to get as1 based on defender graph's nodes/roots
        # Note: This is slightly inefficient but ensures as1 is correct relative to defender's view
        if defender_graph is not attacker_graph:
             _, V_def, as1_def, _, _, _, _, _, _ = generate_game_elements(
                  graph_for_defense_locs, def_virtual_entry_node, def_original_roots
             )
             as1 = as1_def # Use as1 derived from the defender graph

        # Ensure V aligns with the graph nodes used for payoff calculation
        V = sorted(list(graph_for_pathfinding.nodes()), key=str)
        # Recalculate node_order based on this V and graph_for_pathfinding
        try:
            topo_all = list(nx.topological_sort(graph_for_pathfinding))
            node_order = [n for n in topo_all if n in V]
        except nx.NetworkXUnfeasible:
            logger.warning("Graph contains cycles, topological sort not possible. Using sorted node list.")
            node_order = V

    except Exception as e:
         logger.error(f"Error generating game elements: {e}")
         return None

    # --- Baseline Calculation ---
    baseline_attacker_success_prob = 0.0
    shortest_path_found = None

    try:
        # Find the shortest path (using graph_for_pathfinding)
        if not nx.has_path(graph_for_pathfinding, atk_virtual_entry_node, atk_target_node):
            logger.info(f"Baseline: No path exists from {atk_virtual_entry_node} to {atk_target_node}. Attacker success probability = 0.")
            baseline_attacker_success_prob = 0.0
        else:
            shortest_path_found = nx.shortest_path(graph_for_pathfinding,
                                                  source=atk_virtual_entry_node,
                                                  target=atk_target_node,
                                                  weight=None) # Edge count

            logger.info(f"Baseline: Attacker uses shortest path: {shortest_path_found}")
            logger.info(f"Baseline: Defender chooses uniformly from {len(as1)} locations: {as1}")

            if not as1: # No possible defense locations
                logger.info("Baseline: No defense locations possible (as1 is empty).")
                # Calculate attacker success probability if they just run the shortest path unchallenged
                # Need to calculate the outcome U for the shortest path with a "dummy" check node (or no check)
                # Let's use a dummy check unlikely to be on path, or adapt calculation logic
                # For simplicity, assume success prob is 1 if no defenses possible. This might need refinement.
                # Refinement: calculate U with shortest path and no effective check
                # We can simulate this by calculating payoff for a check node not in as1 (won't truncate path)
                # or by adapting the payoff function slightly. Let's adapt:

                # Calculate U for the shortest path with no defender check effect
                U_no_check = calculate_single_payoff_distribution(
                    graph=graph_for_pathfinding, check_node=None, # Indicate no check
                    path=shortest_path_found,
                    V=V, adv_list=adv_list, theta=theta,
                    random_steps_fn=random_steps_fn,
                    attack_rate=attackRate, defense_rate=defenseRate,
                    node_order=node_order
                )
                baseline_attacker_success_prob = U_no_check[-1] if U_no_check is not None else 0.0
                logger.info(f"Baseline: Attacker success probability (no defenses) = {baseline_attacker_success_prob:.4f}")

            else:
                # Calculate expected success probability averaged over defender's uniform choices
                total_success_prob_sum = 0.0
                num_defense_locations = len(as1)

                for check_node in as1:
                    # Calculate the payoff distribution U for this specific check and the shortest path
                    U_for_this_check = calculate_single_payoff_distribution(
                        graph=graph_for_pathfinding, # Use the graph containing the path
                        check_node=check_node,
                        path=shortest_path_found,
                        V=V, adv_list=adv_list, theta=theta,
                        random_steps_fn=random_steps_fn,
                        attack_rate=attackRate, defense_rate=defenseRate,
                        node_order=node_order
                    )

                    if U_for_this_check is not None:
                        prob_reach_target_for_check = U_for_this_check[-1] # Last element is prob of reaching target
                        total_success_prob_sum += prob_reach_target_for_check
                    else:
                        logger.warning(f"Could not calculate payoff for check={check_node}. Skipping.")
                        # Or handle differently? If payoff calc fails, maybe assume attacker fails? Assume 0 for now.
                        total_success_prob_sum += 0.0


                baseline_attacker_success_prob = total_success_prob_sum / num_defense_locations
                logger.info(f"Baseline: Sum of success probabilities over {num_defense_locations} checks = {total_success_prob_sum:.4f}")
                logger.info(f"Baseline: Expected Attacker success probability = {baseline_attacker_success_prob:.4f}")

    except nx.NetworkXNoPath:
        logger.info(f"Baseline: No path found between {atk_virtual_entry_node} and {atk_target_node}. Attacker success probability = 0.")
        baseline_attacker_success_prob = 0.0
    except nx.NodeNotFound as e:
         logger.error(f"Baseline Error: Node not found during pathfinding or element generation: {e}")
         return None
    except Exception as e:
         logger.error(f"An unexpected error occurred during baseline calculation: {e}")
         import traceback
         logger.error(traceback.format_exc())
         return None

    # --- Print and Return Results ---
    print("\n--- REVISED BASELINE CALCULATION RESULT ---")
    print(f"Attacker's Shortest Path: {shortest_path_found}")
    print(f"Defender's Possible Check Locations (as1): {as1} ({len(as1)} locations)")
    print(f"Attacker's Expected Success Probability (Uniform Defender): {baseline_attacker_success_prob:.6f}")
    print("-------------------------------------------\n")

    baseline_result = {
        'optimal_defense': {'baseline_uniform_over_as1': 1.0 / len(as1) if as1 else 0.0},
        'attacker_strategy': {'baseline_shortest_path': 1.0 if shortest_path_found else 0.0},
        'defender_success': baseline_attacker_success_prob, # Using Attacker Success Probability
        'attacker_success': baseline_attacker_success_prob
    }

    logger.info("\n--- REVISED BASELINE CALCULATION SUMMARY ---")
    logger.info(f"Attacker Shortest Path: {shortest_path_found}")
    logger.info(f"Defender Uniform Random Strategy over as1: {as1} ({len(as1)} locations)")
    logger.info(f"Calculated Expected Attacker Success Probability: {baseline_attacker_success_prob:.6f}")
    logger.info("--- END REVISED BASELINE SUMMARY ---\n")

    return baseline_result

In [37]:
# --- Execute Baseline Calculation ---
if __name__ == "__main__":
    print("Starting Baseline Calculation for the defined Attack Graph...")
    print(f"Graph has {attack_graph.number_of_nodes()} nodes and {attack_graph.number_of_edges()} edges.")

    # Define parameters needed by random_steps_fn (example)
    attack_rate_list_param = [2] # Example attack rate
    defense_rate_list_param = [0] # Example defense rate (might not be used by Poisson fn)

    # Call run_game with the defined graph AND the random_steps function
    baseline_result_dict = run_game(
        attacker_graph=attack_graph,
        defender_graph=attack_graph, # Using same graph for baseline
        attack_rate_list=attack_rate_list_param,
        defense_rate_list=defense_rate_list_param,
        random_steps_fn=random_steps # Pass the function here
        )

    if baseline_result_dict:
        print("\nBaseline Calculation Run Complete.")
        final_prob = baseline_result_dict.get('attacker_success', 'N/A')
        print(f"Final Baseline Attacker Success Probability returned: {final_prob}")
    else:
        print("\nBaseline Calculation Failed.")
# --- End Execution ---

INFO: Baseline: Attacker uses shortest path: [1, 2, 3, 'c(6,9)']
INFO: Baseline: Defender chooses uniformly from 6 locations: [2, 3, 4, 5, 7, 8]
INFO: Baseline: Sum of success probabilities over 6 checks = 0.8222
INFO: Baseline: Expected Attacker success probability = 0.1370
INFO: 
--- REVISED BASELINE CALCULATION SUMMARY ---
INFO: Attacker Shortest Path: [1, 2, 3, 'c(6,9)']
INFO: Defender Uniform Random Strategy over as1: [2, 3, 4, 5, 7, 8] (6 locations)
INFO: Calculated Expected Attacker Success Probability: 0.137037
INFO: --- END REVISED BASELINE SUMMARY ---



Starting Baseline Calculation for the defined Attack Graph...
Graph has 9 nodes and 9 edges.

--- REVISED BASELINE CALCULATION RESULT ---
Attacker's Shortest Path: [1, 2, 3, 'c(6,9)']
Defender's Possible Check Locations (as1): [2, 3, 4, 5, 7, 8] (6 locations)
Attacker's Expected Success Probability (Uniform Defender): 0.137037
-------------------------------------------


Baseline Calculation Run Complete.
Final Baseline Attacker Success Probability returned: 0.13703703999999803
